In [7]:
not.installed <- function(pkg) !is.element(pkg, installed.packages()[,1])

if (not.installed("MASS"))  install.packages("MASS")  # we need the MASS package

library(MASS)

#  Read in a table (in csv format) from standard input:
Table = data.matrix(read.csv( file = 'HW0_test.csv', header=TRUE ))
#Table = data.matrix(read.csv( file("stdin"), header=TRUE ))

Distribution = c( "normal", "t", "chi-squared", "lognormal", "exponential", "gamma", "logistic", "weibull", "beta")
  # include other distributions here

Distribution_can_have_negative_values = c( TRUE, TRUE, FALSE, FALSE, FALSE, FALSE, TRUE, FALSE, FALSE)

result = matrix(, nrow = ncol(Table), ncol = 4)
for (j in 1:ncol(Table)) {
    Dataset = Table[,j]        #  j-th dataset = the j-th column of the table
    Dataset_is_nonnegative = !any( Dataset < 0 )
    
    temp_name = ""
    temp_parameter = ""
    temp_likelihood = -Inf
    
    for (i in 1:length(Distribution)) {
        dist_name = Distribution[i]
        if (Distribution_can_have_negative_values[i] || Dataset_is_nonnegative) {
            # do not try to fit a nonnegative distribution to data that is negative

            if (dist_name == "chi-squared") {
                # fitdistr requires special handling of chi-squared
                df = round(mean(Dataset))
                if (df != 0) {
                    fit = suppressWarnings( fitdistr( Dataset, dist_name,
                                  list(df=round(mean(Dataset))), method="BFGS" ) )
                    }
            } 
            else if (dist_name == "beta") {
                if (all(Dataset >= 0) && all(Dataset <= 1) ) {
                    fit = suppressWarnings( fitdistr( Dataset, dist_name,
                                   start=list(shape1=0.5, shape2=0.5) ))
                }
            } 
            else {
                fit = suppressWarnings( fitdistr( Dataset, dist_name ) )
            }

            fitted_parameters = fit$estimate
            log_likelihood = fit$loglik

            parameter_value_string = paste(round(fitted_parameters), collapse=" ")
            # print integer parameters
            #cat(sprintf("%s %s %s\n", dist_name, parameter_value_string, log_likelihood))

            # The optimal distribution is the one with maximum-likelihood
            #  (and:  maximum-likelihood == maximum-log-likelihood).
            # The optimal distribution needs to be tracked here .............
            if (round(log_likelihood) > temp_likelihood) {
                temp_name = dist_name
                temp_parameter = parameter_value_string
                temp_likelihood = round(log_likelihood)
                #cat(sprintf("%s %s %s\n", temp_name, temp_parameter, temp_likelihood))
            }
        }    
    }
    #cat(sprintf("========%s %s %s\n", temp_name, temp_parameter, temp_likelihood))
    result[j,1] = temp_name
    temp_parameter_split = strsplit(temp_parameter, ' ')[[1]]
    result[j,2:(1+length(temp_parameter_split))] = temp_parameter_split
    
    write.table(result, "HW0_test_output.csv", sep=",", row.names=FALSE, col.names=FALSE, quote=FALSE, na="")
}